In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from ipynb.fs.full.utility_functions import gen_data_dict, get_data, get_cumsum, L, trace_back, error_count, write_to_csv, opart

np.random.seed(4)
torch.manual_seed(4)

In [2]:
# Custom loss function
margin = 1
class SquaredHingeLoss(nn.Module):
    def forward(self, predicted, low, high):
        low  = low + margin
        high = high - margin
        loss = torch.relu(low - predicted) + torch.relu(predicted - high)
        return torch.mean(torch.square(loss), dtype=torch.float)

In [3]:
def normalize_data(data):
    mean = data.mean(dim=0)
    std  = data.std(dim=0)
    norm_data = (data - mean) / std
    return norm_data

In [4]:
# data
chosen_feature = ['std_deviation', 'count', 'sum_diff']
data = torch.FloatTensor(pd.read_csv('learning_data/seq_features.csv').iloc[:, 1:][chosen_feature].to_numpy())
data = normalize_data(data)

target_df_1 = pd.read_csv('learning_data/target_lambda_fold1.csv')
target_df_2 = pd.read_csv('learning_data/target_lambda_fold2.csv')

targets_low_1  = target_df_1.iloc[:, 1:2].to_numpy()
targets_high_1 = target_df_1.iloc[:, 2:3].to_numpy()
targets_low_2  = target_df_2.iloc[:, 1:2].to_numpy()
targets_high_2 = target_df_2.iloc[:, 2:3].to_numpy()

targets_low_1  = torch.FloatTensor(targets_low_1)
targets_high_1 = torch.FloatTensor(targets_high_1)
targets_low_2  = torch.FloatTensor(targets_low_2)
targets_high_2 = torch.FloatTensor(targets_high_2)

In [5]:
# data_df   = pd.read_csv('learning_data/seq_features.csv')
# target_df = pd.read_csv('learning_data/target_lambda_bothfold.csv')

# n_train = 300

# # train data
# data = torch.FloatTensor(data_df.iloc[:n_train, 1:].to_numpy())
# data = normalize_data(data)
# targets_1 = torch.FloatTensor(target_df.iloc[:n_train, 1:2].to_numpy())
# targets_2 = torch.FloatTensor(target_df.iloc[:n_train, 2:3].to_numpy())

# # test_data
# data_test = torch.FloatTensor(data_df.iloc[n_train:, 1:].to_numpy())
# data_test = normalize_data(data_test)
# targets_test_1 = torch.FloatTensor(target_df.iloc[n_train:, 1:2].to_numpy())
# targets_test_2 = torch.FloatTensor(target_df.iloc[n_train:, 2:3].to_numpy())

In [6]:
# Define the deep learning model
class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
# Instantiate the model, define custom loss function, and optimizer
model1 = MyModel(input_size = data.shape[1])
model2 = MyModel(input_size = data.shape[1])

squared_hinge_loss = SquaredHingeLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

# Training loop
record_loss1_test = []
record_loss2_test = []
min_loss_1_test = float('inf')
min_loss_2_test = float('inf')
for epoch in range(1000):
    # Forward pass
    outputs1 = model1(data)
    outputs2 = model2(data)
    
    # Compute the custom loss
    loss_1 = squared_hinge_loss(outputs1, targets_low_1, targets_high_1)
    loss_2 = squared_hinge_loss(outputs2, targets_low_2, targets_high_2)
    loss_1_test = squared_hinge_loss(outputs1, targets_low_2, targets_high_2)
    loss_2_test = squared_hinge_loss(outputs2, targets_low_1, targets_high_1)
    
    # Backward pass and optimization
    optimizer1.zero_grad()
    loss_1.backward()
    optimizer1.step()

    optimizer2.zero_grad()
    loss_2.backward()
    optimizer2.step()

    # record
    record_loss1_test.append(loss_1_test.item())
    record_loss2_test.append(loss_2_test.item())

    # save models
    if loss_1_test < min_loss_1_test:
        min_loss_1_test = loss_1_test
        torch.save(model1.state_dict(), 'saved_models/model1_best.pth')
    
    if loss_2_test < min_loss_2_test:
        min_loss_2_test = loss_2_test
        torch.save(model2.state_dict(), 'saved_models/model2_best.pth')
    
    # Print the loss every 100 epochs
    if (epoch) % 100 == 0:
        print(f'Epoch {epoch:5d}, Loss_1: {loss_1.item():8.4f}, Loss_1_test: {loss_1_test.item():8.4f}, Loss_2: {loss_2.item():8.4f}, Loss_2_test: {loss_2_test.item():8.4f}')

Epoch     0, Loss_1:   1.7715, Loss_1_test:   1.8661, Loss_2:   1.8207, Loss_2_test:   1.2023
Epoch   100, Loss_1:   1.3177, Loss_1_test:   1.5205, Loss_2:   1.5279, Loss_2_test:   1.0895
Epoch   200, Loss_1:   1.0983, Loss_1_test:   1.3473, Loss_2:   1.4283, Loss_2_test:   1.0693
Epoch   300, Loss_1:   1.0313, Loss_1_test:   1.3128, Loss_2:   1.3716, Loss_2_test:   1.0608
Epoch   400, Loss_1:   1.0221, Loss_1_test:   1.3212, Loss_2:   1.3277, Loss_2_test:   1.0527
Epoch   500, Loss_1:   1.0188, Loss_1_test:   1.3240, Loss_2:   1.2875, Loss_2_test:   1.0479
Epoch   600, Loss_1:   1.0161, Loss_1_test:   1.3234, Loss_2:   1.2679, Loss_2_test:   1.0510
Epoch   700, Loss_1:   1.0142, Loss_1_test:   1.3248, Loss_2:   1.2595, Loss_2_test:   1.0565
Epoch   800, Loss_1:   1.0136, Loss_1_test:   1.3251, Loss_2:   1.2534, Loss_2_test:   1.0608
Epoch   900, Loss_1:   1.0136, Loss_1_test:   1.3239, Loss_2:   1.2476, Loss_2_test:   1.0638


In [8]:
# Load model1
model1 = MyModel(input_size=data.shape[1])
model1.load_state_dict(torch.load('saved_models/model1_best.pth'))
model1.eval()  # Set the model to evaluation mode

# Load model2
model2 = MyModel(input_size=data.shape[1])
model2.load_state_dict(torch.load('saved_models/model2_best.pth'))
model2.eval()  # Set the model to evaluation mode


MyModel(
  (fc1): Linear(in_features=3, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)

In [9]:
print("loss for fold 2: ",squared_hinge_loss(model1(data), targets_low_2, targets_high_2).item())
print("loss for fold 1: ",squared_hinge_loss(model2(data), targets_low_1, targets_high_1).item())

loss for fold 2:  1.3113869428634644
loss for fold 1:  1.047221064567566


In [10]:
with torch.no_grad():
    ldas1 = model1(data).numpy().reshape(-1)
    ldas2 = model2(data).numpy().reshape(-1)

In [11]:
seqs   = gen_data_dict('sequence_label_data/signals.gz')
labels = gen_data_dict('sequence_label_data/labels.gz')

header = ['sequenceID', 'fold_1_total_labels', 'fold_2_total_labels', 'fold_1_errs', 'fold_2_errs']

for i in range(len(seqs)):
    # generate data
    sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(i, seqs=seqs, labels=labels)
    sequence_length = len(sequence)-1

    # vectors of cumulative sums
    y, z = get_cumsum(sequence)

    # get total labels
    fold1_total_labels = len(neg_start_1) + len(pos_start_1)
    fold2_total_labels = len(neg_start_2) + len(pos_start_2)

    # run each lambda and record it into csv file
    row  = [i, fold1_total_labels, fold2_total_labels]

    chpnt_fold1 = opart(10**ldas2[i], sequence, y, z)
    chpnt_fold2 = opart(10**ldas1[i], sequence, y, z)

    err_1 = error_count(chpnt_fold1, neg_start_1, neg_end_1, pos_start_1, pos_end_1)
    err_2 = error_count(chpnt_fold2, neg_start_2, neg_end_2, pos_start_2, pos_end_2)
    
    row.append(sum(err_1))
    row.append(sum(err_2))

    write_to_csv('learning_output/deep_2.csv', header, row)